In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = tf.test.gpu_device_name()
print(device)

/device:GPU:0


## Linear Regression with tensorflow keras API

In [12]:
l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

# the relationship of x and y is Y = 2X - 1
x = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
y = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(x, y, epochs=500, verbose=0) # verbose=0 to hide epochs log
print(model.predict([10.0]))
print('Here is what i learn: {}'.format(l0.get_weights()))
# Y = WX - B
# Y = 1.9965292X - 0.9892394
# it's close to Y = 2X - 1

1/1 [==============================] - 0s 68ms/step
[[18.976053]]
Here is what i learn: [array([[1.9965292]], dtype=float32), array([-0.9892394], dtype=float32)]


## Computer vision
train model by image pixels

In [5]:
data = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = data.load_data()

# แปลง greyscale ระหว่าง 0 - 255 ให้เหลือแค่ 0 - 1 (ด้วยการหาร 255)
train_images = train_images / 255.0
test_images = test_images / 255.0

4422102/4422102 [==============================] - 1s 0us/step


In [38]:
class myCallback(tf.keras.callbacks.Callback):
    # Override on_epoch_end method to stop traning when accuracy reached 95%
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.95:
            print('\nReached 95% accuracy so cancelling training!')
            self.model.stop_training = True

callbacks = myCallback()

model = Sequential([
    Flatten(input_shape=(28, 28)), # แปลงรูปภาพขนาด 28x28 ให้กลายเป็น layer เดียวด้วยการ flatten
    Dense(128, activation=tf.nn.relu), # 128 = จำนวน neural ที่ต้องการใน layer
    Dense(10, activation=tf.nn.softmax), # layer คำตอบมี 10 แบบ (เลข 0 - 9)
])

model.compile(
    optimizer='adam', # นิยมใช้กับ CNN ประมวลผลภาพ
    loss='sparse_categorical_crossentropy', # cross entropy นิยมใช้กับประเภท classification (จำแนกประเภท)
    metrics=['accuracy']
)

model.fit(train_images, train_labels, epochs=50, callbacks=[callbacks])
model.evaluate(test_images, test_labels)

Epoch 1/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.5038 - accuracy: 0.8238
Epoch 2/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3778 - accuracy: 0.8647
Epoch 3/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3376 - accuracy: 0.8768
Epoch 4/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3128 - accuracy: 0.8837
Epoch 5/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2956 - accuracy: 0.8903
Epoch 6/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2831 - accuracy: 0.8947
Epoch 7/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2686 - accuracy: 0.8989
Epoch 8/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2570 - accuracy: 0.9043
Epoch 9/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2471 - accuracy: 0.9073
Epoch 10/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.239

[0.42016878724098206, 0.8916000127792358]

In [39]:
classifications = model.predict(test_images)
print(classifications[0].shape) # shape = 10 เพราะคำตอบมี 10 แบบ (0-9) ซึ่ง index ที่ 9 (เลข 9) มีค่ามากสุดจึงตอบ 9
print(np.argmax(classifications[0]))
print(test_labels[0])

313/313 [==============================] - 1s 2ms/step
(10,)
9
9


## Convolutional neural network (CNN)
filter images down to constitent element instead of pixels

In [7]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2), # ลดขนาดรูปภาพโดยการเอาค่าสี pixels ที่มากที่สุด
    Conv2D(64,(3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation=tf.nn.relu),
    Dense(10, activation=tf.nn.softmax) # softmax นิยมใช้กับ classification แบ่งประเภท
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train_images, train_labels, epochs=5)
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 17s 4ms/step - loss: 0.4459 - accuracy: 0.8382
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2978 - accuracy: 0.8906
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2509 - accuracy: 0.9069
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2177 - accuracy: 0.9197
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.2755 - accuracy: 0.9059


[0.27545616030693054, 0.9059000015258789]

In [9]:
classifications = model.predict(test_images)
print(np.argmax(classifications[1]))
print(test_labels[1])

313/313 [==============================] - 1s 2ms/step
2
2
